In [1]:
# import required libraries
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

In [2]:
# load and read dataset
dataset = pd.read_excel('final_dataset.xlsx')
dataset.head()

,Red_Mean,Red_Median,Red_Variance,Red_Std,Green_Mean,Green_Median,Green_Variance,Green_Std,Blue_Mean,Blue_Median,Blue_Variance,Blue_Std,label
0,121.849594,145.0,3304.497718,57.484761,125.967102,135.0,1763.033006,41.988487,111.251572,134.0,3330.577657,57.711157,0
1,103.830032,102.0,2152.241390,46.392256,114.606018,121.0,1838.561514,42.878450,95.103455,88.0,3073.705972,55.441013,1
2,116.047089,134.0,2488.556041,49.885429,119.544250,125.0,1193.293666,34.544083,100.909332,119.0,2621.018971,51.195888,0
3,136.331741,139.0,1649.347666,40.612162,147.529755,151.0,1412.353454,37.581291,126.857681,129.0,2512.158838,50.121441,1
4,129.850250,134.0,1940.935613,44.056051,131.718399,139.0,1782.426972,42.218799,101.312119,94.0,2798.758005,52.903289,1


In [3]:
# separate dataset into dependent and independent sets
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]
y.head()

0    0
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [4]:
# scale the values
scaler = StandardScaler()
x = scaler.fit_transform(x)
x

array([[-0.23500032,  0.42004827,  1.22358984, ...,  0.69557054,
         0.52964155,  0.57699777],
       [-1.07910805, -1.23470125, -0.25989263, ..., -1.1143668 ,
         0.21483548,  0.28083278],
       [-0.50681269, -0.00325975,  0.1730986 , ...,  0.10537358,
        -0.33994975, -0.27299009],
       ...,
       [ 0.13847124,  0.15067044,  0.22896819, ...,  0.26275944,
         0.22765812,  0.29313266],
       [-1.74647231, -1.54256162, -1.5399771 , ..., -1.07502033,
        -1.67287985, -1.84339441],
       [-0.2624165 , -0.31112012, -1.30148167, ..., -0.1307052 ,
        -1.07766581, -1.0899189 ]])

In [5]:
# split dataset into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [6]:
# prepare model
model = Sequential()

# input layer for ann
model.add(Dense(units = 12, input_dim = 12, input_shape = (12, ), kernel_initializer = 'uniform'))

# first hidden layer
model.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))

# second hidden layer
model.add(Dense(units = 128, activation = 'tanh', kernel_initializer = 'uniform'))

# third hidden layer
model.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))

# output layer
model.add(Dense(units = 3, activation = 'softmax', kernel_initializer = 'uniform'))

# summary for model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                156       
                                                                 
 dense_1 (Dense)             (None, 128)               1664      
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 35231 (137.62 KB)
Trainable params: 35231 (137.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# compile the model
model.compile(optimizer = 'adam', loss = 'SparseCategoricalCrossentropy', metrics = ['accuracy'])

In [8]:
# train model
model.fit(x_train, y_train, epochs = 100, batch_size = 20)

Epoch 1/100


229/229 [==============================] - 4s 4ms/step - loss: 0.6747 - accuracy: 0.7139
Epoch 2/100
229/229 [==============================] - 1s 4ms/step - loss: 0.2667 - accuracy: 0.9156
Epoch 3/100
229/229 [==============================] - 1s 4ms/step - loss: 0.2092 - accuracy: 0.9292
Epoch 4/100
229/229 [==============================] - 1s 4ms/step - loss: 0.1600 - accuracy: 0.9371
Epoch 5/100
229/229 [==============================] - 1s 4ms/step - loss: 0.1365 - accuracy: 0.9478
Epoch 6/100
229/229 [==============================] - 1s 4ms/step - loss: 0.1269 - accuracy: 0.9546
Epoch 7/100
229/229 [==============================] - 1s 4ms/step - loss: 0.1083 - accuracy: 0.9619
Epoch 8/100
229/229 [==============================] - 1s 4ms/step - loss: 0.1001 - accuracy: 0.9640
Epoch 9/100
229/229 [==============================] - 1s 4ms/step - loss: 0.0947 - accuracy: 0.9656
Epoch 10/100
229/229 [==============================] - 1s 4ms/step - loss: 0.0956 - accu

# Hyperparameter Tunning

### Tuning Optimizer

In [9]:
# select appropriate optimizer by hyperparameter tuning
def build_model(hp):
    model1 = Sequential()

    model1.add(Dense(units = 12, input_dim = 12, input_shape = (12, ), kernel_initializer = 'uniform'))
    model1.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))
    model1.add(Dense(units = 128, activation = 'tanh', kernel_initializer = 'uniform'))
    model1.add(Dense(units = 128, activation = 'relu', kernel_initializer = 'uniform'))
    model1.add(Dense(units = 3, activation = 'softmax', kernel_initializer = 'uniform'))

    optimizer = hp.Choice('optimizer', values = ['adam', 'rmsprop', 'adadelta', 'SGD', 'adamw'])

    model1.compile(optimizer = optimizer, loss = 'SparseCategoricalCrossentropy', metrics = ['accuracy'])

    return model1

In [10]:
# now make a tuner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)

In [11]:
# now search for the best optimizer
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.9631901979446411
Total elapsed time: 00h 00m 40s


In [12]:
# now find the best optimizer
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [13]:
# fetch best model
model2 = tuner.get_best_models(num_models = 1)[0]

In [14]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                156       
                                                                 
 dense_1 (Dense)             (None, 128)               1664      
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 35231 (137.62 KB)
Trainable params: 35231 (137.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Tuning Number of Neurons

In [23]:
# Tuning number of neurons
def no_neurons(hp):

    # initiate model
    model = Sequential()

    # range of no of neurons
    unit = hp.Int('unit', min_value = 12, max_value = 128, step = 8)

    # input layer
    model.add(Dense(units = 12, input_dim = 12, input_shape = (12, ), kernel_initializer = 'uniform'))

    # first hidden layer
    model.add(Dense(units = unit, activation = 'relu', kernel_initializer = 'uniform'))

    # second hidden layer
    model.add(Dense(units = unit, activation = 'tanh', kernel_initializer = 'uniform'))

    # third hidden layer
    model.add(Dense(units = unit, activation = 'relu', kernel_initializer = 'uniform'))

    model.compile(optimizer = 'adam', loss = 'SparseCategoricalCrossentropy', metrics = ['accuracy'])

    return model

In [24]:
tuner1 = kt.RandomSearch(no_neurons, objective = 'val_accuracy', max_trials = 16, directory = 'Arfan')

In [25]:
tuner1.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 15 Complete [00h 00m 06s]
val_accuracy: 0.9070990085601807

Best val_accuracy So Far: 0.945661723613739
Total elapsed time: 00h 01m 35s


In [22]:
tuner1.get_best_hyperparameters()[0].values

{'unit': 108}

In [26]:
model12 = tuner1.get_best_models(num_models = 1)[0]
model12.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                156       
                                                                 
 dense_1 (Dense)             (None, 100)               1300      
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
Total params: 21656 (84.59 KB)
Trainable params: 21656 (84.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Tuning Number of Layers

In [27]:
def num_layers(hp):
    model22 = Sequential()

    # input layer
    model22.add(Dense(units = 12, input_dim = 12, input_shape = (12, ), kernel_initializer = 'uniform'))

    # apply for loop to get exact number of hidden layers
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        model22.add(Dense(units = 108, activation = 'relu', kernel_initializer = 'uniform'))
    model22.add(Dense(units = 3, activation = 'softmax', kernel_initializer = 'uniform'))

    # compile
    model22.compile(optimizer = 'adam', loss = 'SparseCategoricalCrossentropy', metrics = ['accuracy'])

    return model22

In [28]:
tuner22 = kt.RandomSearch(num_layers, objective = 'val_accuracy',  max_trials = 10, directory = 'tuner22')

In [29]:
tuner22.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.32953548431396484

Best val_accuracy So Far: 0.9465381503105164
Total elapsed time: 00h 00m 35s


In [30]:
tuner22.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [31]:
modell = tuner22.get_best_models(num_models = 1)[0]
modell.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                156       
                                                                 
 dense_1 (Dense)             (None, 108)               1404      
                                                                 
 dense_2 (Dense)             (None, 108)               11772     
                                                                 
 dense_3 (Dense)             (None, 108)               11772     
                                                                 
 dense_4 (Dense)             (None, 3)                 327       
                                                                 
Total params: 25431 (99.34 KB)
Trainable params: 25431 (99.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
